In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

ROOT = Path('../')
DATA_DIR = ROOT/'data'

CRS_NZGD49 = {'init': 'epsg:27200', 'no_defs': True}
CRS_NZTM = {'init': 'epsg:2193', 'no_defs': True}
CRS_WGS84 = {'init': 'epsg:4326'}

%matplotlib inline

In [ ]:
# Prepare area unit table

path = DATA_DIR/'raw'/'Geographical Table.csv'
f = pd.read_csv(path, dtype={'SAU': str})
f = f.rename(columns={
    'SAU': 'au2001', 
    'SAU.Desc': 'au_name', 
    'TA': 'territory',
    'Region': 'region',
})
del f['Water']
f.head()

path = DATA_DIR/'raw'/'Market Rent Areas.csv'
g = pd.read_csv(path, dtype={'SAU': str})
g = g.rename(columns={
    'SAU': 'au2001', 
    'MARKET RENT DESCRIPTION': 'rental_area',
    'TA': 'territory',
    'AU NAME': 'au_name',
})

# Clean rental areas
def clean(x):
    y = x.split(' - ')
    y = y[1] if 'District' not in y[1] else y[0]
    return y

g['rental_area'] = g['rental_area'].map(clean)


f = f.merge(g[['au2001', 'rental_area']])

path = DATA_DIR/'au2001.csv'
f.to_csv(str(path), index=False)
f.head()

# Prepare geodata as GeoJSON

In [ ]:
# Read Shapefile

path = DATA_DIR/'raw'/'NZ_AU01_region_simplified'/'NZ_AU01_region.shp'
au = gpd.read_file(str(path))
au.crs = CRS_NZGD49
au = au.to_crs(CRS_WGS84)
au = au.rename(columns={'AU01': 'au2001', 'AU_DESC': 'au_name'})
print(au.shape)
print(au.head())
au.head().plot()


In [ ]:
# Remove water area units

pattern = r'ocean|strait|inlet|harbour'
cond = au['au_name'].str.contains(pattern, case=False)
au = au[~cond].copy()
print(au.shape)
au.head().plot()


In [ ]:
# Merge geodata and metadata, drop null regions, and write to file

path = DATA_DIR/'au2001.csv'
f = pd.read_csv(path, dtype={'au2001': str})

g = au.merge(f[['au2001', 'territory', 'region', 'rental_area']])
g = g[g['region'].notnull()].copy()

path = DATA_DIR/'au2001.geojson'
with path.open('w') as tgt:
    tgt.write(g.to_json())

g.head()

# Create geodata for rental areas 

In [ ]:
# Dissolve area units by area unit group

path = DATA_DIR/'au2001.geojson'
au = gpd.read_file(str(path))

ra = au[['rental_area', 'region', 'territory', 'geometry']].dissolve(by='rental_area').reset_index()

path = DATA_DIR/'rental_areas.geojson'
with path.open('w') as tgt:
    tgt.write(ra.to_json())

ra.head()

# Prepare rent data

In [ ]:
# Reshape and merge all rent data sets

def clean(f, name):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=name)
    
    return f

paths = [
    DATA_DIR/'raw'/'Detailed Bonds Lodged.csv',
    DATA_DIR/'raw'/'Detailed Mean Rents.csv',
    DATA_DIR/'raw'/'Detailed Geomean Rents.csv',
    DATA_DIR/'raw'/'Detailed Synthetic Lower Quartile Rents.csv',
    DATA_DIR/'raw'/'Detailed Synthetic Upper Quartile Rents.csv',
]
names = ['rent_count', 'rent_mean', 'rent_geo_mean', 'rent_synthetic_lower_quartile', 'rent_synthetic_upper_quartile']
frames = []
for path, name in zip(paths, names):
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, name))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = DATA_DIR/'au2001.csv'
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = DATA_DIR/'rents.csv'
f.to_csv(str(path), index=False)
f[f['rent_count'].notnull()].head()

# Explorer rents

In [ ]:
path = DATA_DIR/'rents.csv'
f = pd.read_csv(path, dtype={'au2001': str})
f.head()


In [ ]:
# Slice in time and aggregate 

def aggregate_rents(f, date, groupby_cols=('rental_area', '#bedrooms')):
    """
    """
    cond = f['quarter'] >= date
    f = f[cond].copy()
    
    def my_agg(group):
        d = {}
        d['territory'] = group['territory'].iat[0]
        d['region'] = group['region'].iat[0]
        d['rent_count'] = group['rent_count'].sum()
        d['rent_mean'] = (group['rent_mean']*group['rent_count']).sum()/d['rent_count']
        d['rent_geo_mean'] = (group['rent_geo_mean']**(group['rent_count']/d['rent_count'])).prod()
        return pd.Series(d)

    g = f.groupby(groupby_cols).apply(my_agg).reset_index()
    return g

agg_rents = aggregate_rents(f, '2016-12-01')
agg_rents

In [ ]:
cond = agg_rents['region'] == 'Canterbury'
a = agg_rents[cond].copy()

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_count'].dropna().shape[0]/group['rent_count'].shape[0]
    return pd.Series(d)

a.groupby('#bedrooms').apply(hits).reset_index()

# Choose representative points for rental areas using property titles

In [ ]:
path = DATA_DIR/'rental_areas.geojson'
ra = gpd.read_file(str(path))

path = DATA_DIR/'property_titles.geojson'
t = gpd.read_file(str(path))
t.head()

In [ ]:
%time f = gpd.sjoin(t[['geometry', 'fid']], ra, op='intersects')
f.head()

In [ ]:
def pt(group):
    d = {}
    d['geometry'] = so.unary_union(group['geometry']).representative_point()
    d['territory'] = group['territory'].iat[0]
    d['region'] = group['region'].iat[0]
    return pd.Series(d)

g = gpd.GeoDataFrame(f.groupby('rental_area').apply(pt).reset_index())

path = DATA_DIR/'rental_area_points.geojson'
with path.open('w') as tgt:
    tgt.write(g.to_json())

g.head()

In [ ]:
g[g['region'] == 'Auckland']

# Use Google Maps to compute times and distance matrices




In [24]:
def get_secret(key, secrets_path=ROOT/'secrets.json'):
    secrets_path = Path(secrets_path)
    with secrets_path.open() as src:
        secrets = json.load(src)
    return secrets[key]

GOOGLE_MATRIX_URL = "https://maps.googleapis.com/maps/api/distancematrix/json"
GOOGLE_KEY = get_secret('GOOGLE_API_KEY_PHIL')

def get_matrix(origins, destinations, mode, departure_time=None, url=GOOGLE_MATRIX_URL, key=GOOGLE_KEY, 
  timezone='Pacific/Auckland'):
    """
    Call Google to compute the duration-distance matrix from the list of origins to the list of destinations
    by the given mode at the given departure time.
    
    INPUT:
        origins
            List of WGS84 longitude-latitude pairs that will be round to 5 decimal places 
        destinations
            List of WGS84 longitude-latitude pairs that will be round to 5 decimal places 
        mode
            String; one of 'driving', 'walking', 'bicycling', or 'transit'
        departure_time
            Optional; string; departure datetime as a string of the form '%Y-%m-%d %H:%M' or the string 'now'; 
            can't be from the past
        timezone
            String; timezone for query, e.g. 'Pacific/Auckland'; see https://en.wikipedia.org/wiki/List_of_tz_database_time_zones 
            
    OUTPUT:
        A decoded JSON string described at https://developers.google.com/maps/documentation/distance-matrix/intro#DirectionsResponseElements .
    NOTES:
        - If ``departure_time`` is not null and ``mode='driving'``, then each request must contain at most 100 elements, 
          where the number of elements equals the product of the number of origins and number of destinations.
    """
    valid_modes = ['driving', 'walking', 'bicycling', 'transit']
    if mode not in valid_modes:
        raise ValueError('mode must be one of {!s}'.format(valid_modes))
    
    origs = '|'.join(["{:.05f},{:.05f}".format(lat, lon) for lon, lat in origins])
    dests = '|'.join(["{:.05f},{:.05f}".format(lat, lon) for lon, lat in destinations])
    if departure_time not in [None, 'now']:
        tz = pytz.timezone(timezone)
        departure_time = dt.datetime.strptime(departure_time, '%Y-%m-%d %H:%M')
        departure_time = int(tz.localize(departure_time).timestamp())
        
    params = {
        'origins': origs,
        'destinations': dests,
        'key': key,
        'mode': mode,
        'departure_time': departure_time,
    }
    r = requests.get(url, params=params)

    # Raise an error if bad request
    r.raise_for_status()

    return r.json()         

def matrix_to_df(matrix, orig_names=None, dest_names=None):
    """
    Given a (decoded) JSON time-distance matrix of the form output by :func:``get_matrix``, 
    a list of origin names (defaults to [0, 1, 2, etc.]), 
    and a list of destination names (defaults to [0, 1, 2, etc.]), convert the matrix to a DataFrame with
    the columns:
    
    - ``'origin'``: one of ``orig_names``
    - ``'destination'``: one of ``dest_names``
    - ``'duration'``: time from origin to destination
    - ``'distance'``: distance from origin to destination
    
    The origin and destination names should be listed in the same order as the 'sources' and 'targets' 
    attributes of ``matrix``, respectively.
    """
    # Initialize DataFrame
    columns = ['orig', 'orig_name', 'dest', 'dest_name', 'duration', 'distance']
    f = pd.DataFrame([], columns=columns)
    
    # Append origins and destinations
    origs, dests =  zip(*product(matrix['origin_addresses'], matrix['destination_addresses']))
    f['orig'] = origs
    f['dest'] = dests
    if orig_names is not None and dest_names is not None:
        orig_names, dest_names = zip(*product(orig_names, dest_names))
        f['orig_name'] = orig_names
        f['dest_name'] = dest_names
        
    # Append durations and distances
    if 'duration_in_traffic' in matrix['rows'][0]['elements'][0]:
        dur_key = 'duration_in_traffic'
    else:
        dur_key = 'duration'
    durs, dists = zip(*[(e[dur_key]['value'], e['distance']['value'])
      for r in matrix['rows'] for e in r['elements']])
    f['duration'] = durs
    f['distance'] = dists

    return f

def build_matrix(rental_area_points, mode, departure_time=None, chunk_size=100, 
  url=GOOGLE_MATRIX_URL, key=GOOGLE_KEY):
    """
    Compute the duration-distance matrix between all pairs of rental area points given,
    but skip the diagonal entries, that is, the ones with origin equal to destination.
    To do this, call:func:`get_matrix` repeatedly.
    Group the duration-distance calls into ``chunk_size``-to-1 chunks. 
    
    INPUT:
        rental_area_points
            GeoDataFrame
        mode
            See :func:`get_matrix`
        departure_time
            See :func:`get_matrix`
        chunk_size
            Max number of origin-destination rows per matrix query
        url
            See :func:`get_matrix`
        key
            See :func:`get_matrix`
            
    OUTPUT:
        A DataFrame of the form...
    """
    f = rental_area_points.copy()
    frames = []
    for __, row in f.iterrows():
        dests = [row['geometry'].coords[0]]  # One destination
        ra = row['rental_area']
        dest_names = [ra]
        
        # Chunk points and get matrix for each chunk to destination
        ff = f[f['rental_area'] != ra].copy()
        num_chunks = math.ceil(ff.shape[0]/chunk_size)
        for g in np.array_split(ff, num_chunks):
            # Get origins
            origs = [geo.coords[0] for geo in g['geometry']] 
            orig_names = g['rental_area'].values 
            # Get OD matrix
            j = get_matrix(origs, dests, mode=mode, departure_time=departure_time, url=url, key=key)
            df = matrix_to_df(j, orig_names, dest_names)
#             try:
#                 j = get_matrix(origs, dests, mode=mode, departure_time=departure_time, url=url, key=key)
#                 df = matrix_to_df(j, orig_names, dest_names)
#             except:
#                 df = pd.DataFrame()
#                 df['orig'] = np.nan
#                 df['orig_name'] = orig_names
#                 df['dest'] = np.nan
#                 df['dest_name'] = ra
#                 df['duration'] = np.nan
#                 df['distance'] = np.nan
            frames.append(df)
            
    return pd.concat(frames).sort_values(['orig', 'dest'])


In [14]:
# Test some
origs = [
    [174.66339111328125, -36.45000844447082], 
    [174.76158142089844, -36.86533886128865],
    [174.85633850097656, -37.20517535620264],
]
dests = origs[:2]
matrix = get_matrix(origs, dests, mode='driving', departure_time='2017-06-01 08:00')
matrix_to_df(matrix, ['bingo', 'bongo', 'boom'], ['bingo', 'bongo'])

,orig,orig_name,dest,dest_name,time,distance,duration
0,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,NaN,0,0
1,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,NaN,56385,3685
2,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,NaN,53674,2813
3,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,NaN,0,8
4,"293-365 Patumahoe Rd, Pukekohe 2120, New Zealand",boom,"173 Cowan Bay Rd, Warkworth 0983, New Zealand",bingo,NaN,104203,6499
5,"293-365 Patumahoe Rd, Pukekohe 2120, New Zealand",boom,"23-25 Mount Eden Rd, Grafton, Auckland 1023, N...",bongo,NaN,51451,4270


In [3]:
path = DATA_DIR/'rental_area_points.geojson'
f = gpd.read_file(str(path))
f = f[f['region'] == 'Auckland'].copy()
f

,geometry,id,region,rental_area,territory
2,POINT (174.7136142163085 -36.71740768433499),2,Auckland,Albany,North Shore City
6,POINT (174.689014050193 -36.89612845095292),6,Auckland,Avondale,Auckland City
10,POINT (174.7513445789724 -36.89353312125105),10,Auckland,Balmoral,Auckland City
13,POINT (174.6988893987124 -36.79717233861077),13,Auckland,Beachhaven/Birkdale,North Shore City
18,POINT (174.7059674167557 -36.91481011710743),18,Auckland,Blockhouse Bay/New Windsor,Auckland City
19,POINT (174.9213767017827 -36.91701523623668),19,Auckland,Botony Downs,Manukau City
21,POINT (174.7347763337456 -36.71729061740317),21,Auckland,Browns Bay,North Shore City
22,POINT (174.9105111501434 -36.87507566798835),22,Auckland,Bucklands Beach,Manukau City
30,POINT (174.7686467667584 -36.84997406733503),30,Auckland,Central East,Auckland City
34,POINT (174.7603149671217 -36.85150535125068),34,Auckland,Central West,Auckland City


In [25]:
# Test some more
departure_time = '2017-06-01 07:30'
m = build_matrix(f[:2], mode='driving', departure_time=departure_time)
m

,orig,orig_name,dest,dest_name,duration,distance
0,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",Albany,"34 Nacton Ln, Avondale, Auckland 1026, New Zea...",Avondale,2550,29972
0,"34 Nacton Ln, Avondale, Auckland 1026, New Zea...",Avondale,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",Albany,2061,29340


In [ ]:
# Estimate cost of query at 0.5/1000 USD/query over 2500 queries
def compute_google_cost(n):
    n = f.shape[0]
    d = {}
    d['#rental areas'] = n
    N = 4*n**2
    d['#queries needed for 4 modes'] = N
    d['price for queries in USD'] = (N - 2500)*(0.5/1000)
    d['exceeds 100000-query daily limit?'] = N > 100000 
    return pd.Series(d)

compute_google_cost(f.shape[0])

In [ ]:
# Multimodal time-distance matrices not implemented by Mapzen

departure_time='2017-06-01 07:30'
for mode in ['driving', 'walking', 'bicycling', 'transit']:
    %time m = compute_all_pairs_matrix(f, mode=mode, departure_time=departure_time)
    print(m.head())
    path = DATA_DIR/'auckland'/'{!s}_matrix.csv'.format(mode)
    m.to_csv(str(path), index=False)

In [ ]:
matrix